In [1]:
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"../Algorithm encapsulation"))
sys.path.append(os.path.join(os.getcwd(),".."))
print(sys.path)
from MXband import MXband
import pandas as pd
from ga_platoon import max_dt

['c:\\Users\\Administrator\\Desktop\\速度爬取\\Case Analysis', 'd:\\Anaoconda\\envs\\cplex\\python37.zip', 'd:\\Anaoconda\\envs\\cplex\\DLLs', 'd:\\Anaoconda\\envs\\cplex\\lib', 'd:\\Anaoconda\\envs\\cplex', '', 'C:\\Users\\Administrator\\AppData\\Roaming\\Python\\Python37\\site-packages', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\requests-2.22.0-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\docloud-1.0.375-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\docplex-2.11.176-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\chardet-3.0.4-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\idna-2.8-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\urllib3-1.25.11-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\cplex-12.10.0.0-py3.7.egg', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\win32', 'd:\\Anaoconda\\envs\\cplex\\lib\\site-packages\\win32\\lib', 'd:\\Anaocond

In [2]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [3]:
phase = np.loadtxt("./data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("./data/volume")
qv = np.loadtxt("./data/path_vol")
phase_get = np.load("./data/phase_get.npy")
d = np.loadtxt("./data/distance")
straight_get = np.load("./data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 4, 10
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 1e6
nx = 0.00000001
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","下行速度数据.csv"),index_col=0)

p=[0,0,0,0,1,0,0,0,1,0,1,0,0]

In [4]:
mb=MXband(phase,[80,180],straight_get,d,lower,upper,speedVar[0], speedVar[1],vol,px)
mb._solve()

conflict(s): 0
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125

Multi-objective solve log . . .

Index  Priority  Blend          Objective      Nodes  Time (sec.)  DetTime (ticks)
    1         0      3   1.1870433333e+03          0         0.05            16.62
status  = multi-objective optimal
time    = 0.047 s.
problem = MILP
gap     = 5.0951e+73%

object value: 1963


In [7]:
a=mb.get_dataframe()
a

,cross_number,distance,sg1,sg2,b1,b2,offset,p,t1,t2,z,u1,u2,w1,car_t1,w2,car_t2
0,1.0,800.0,0.282,0.330,50.76,59.40,0.00,0.0,200.00,165.78,180.0,0.0,0.0,0.141000,3.907985e-14,0.165000,50.76
1,2.0,520.0,0.426,0.368,33.66,36.00,0.00,0.0,64.47,130.00,180.0,0.0,0.0,0.252111,2.855000e+01,0.100000,76.68
2,3.0,500.0,0.187,0.200,33.66,36.00,93.02,0.0,111.35,105.62,180.0,0.0,0.0,0.093500,9.302000e+01,0.100000,126.68
3,4.0,490.0,0.277,0.277,45.90,36.00,14.29,0.0,122.50,53.54,180.0,0.0,0.0,0.149500,1.825000e+01,0.137611,21.06
4,5.0,370.0,0.255,0.255,45.90,42.26,140.75,0.0,37.00,37.00,180.0,0.0,0.0,0.127500,1.407500e+02,0.137611,144.39
5,6.0,254.0,0.650,0.650,43.74,43.74,106.65,0.0,63.50,44.32,180.0,0.0,0.0,0.522500,1.788300e+02,0.121500,106.65
6,7.0,585.0,0.243,0.243,29.70,43.74,62.33,0.0,110.65,58.50,180.0,0.0,0.0,0.121500,6.935000e+01,0.121500,62.33
7,8.0,1020.0,0.165,0.165,29.70,8.00,0.00,0.0,198.32,171.41,180.0,0.0,0.0,0.082500,0.000000e+00,0.142778,21.70
8,9.0,409.0,0.298,0.298,53.64,51.40,6.35,0.0,102.25,68.88,180.0,0.0,0.0,0.149000,6.350000e+00,0.155222,8.59
9,10.0,547.0,0.379,0.379,68.22,48.24,101.31,0.0,135.29,54.70,180.0,0.0,0.0,0.189500,1.013100e+02,0.245000,121.29


In [6]:
a[["b1","b2"]].mean()

b1    48.544615
b2    46.229231
dtype: float64